# Notebooks pour Allgo
[Allgo](https://allgo.inria.fr/) est un service qui offre une interface web pour des outils en ligne de commande. Il dispense ainsi les utilisateurs d'installer les programmes en question et offre la possibilité de cacher leur code source et modèles (pour les outils ayant besoin d'un entraînement). Ses objectifs sont (probablement) de 
* Permettre l'évaluation d'outils de recherche par des groupes de recherche et des industriels
* Archiver un environnement d'exécution

Allgo offre donc un gain de reproductibilité et cause une perte d'ouverture.

## Speads

[Speads](https://allgo.inria.fr/app/speads) est un exemple d'application disponible sur Allgo. Speads segmente une conversation, identifie des locuteurs et tente de déterminer leur genre.

In [5]:
import csv


In [6]:
with open('conv1_speads.tsv') as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        print(row)

['speaker0_F', '0.00000', '1.09000']
['speaker0_F', '1.65000', '2.60000']
['speaker1_M', '3.07000', '4.56000']
['speaker2_M', '7.35000', '20.74000']
['speaker0_F', '20.74000', '23.20000']
['speaker3_F', '23.20000', '24.73000']
['speaker0_F', '24.73000', '27.33000']
['speaker4_F', '29.98000', '41.44000']
['speaker3_F', '41.44000', '43.91000']
['speaker1_M', '46.53000', '75.32000']
['speaker0_F', '76.14000', '78.75000']
['speaker0_F', '79.98000', '90.58000']
['speaker4_F', '90.58000', '97.64000']
['speaker3_F', '97.64000', '129.35001']
['speaker4_F', '129.35001', '134.85001']
['speaker3_F', '134.85001', '152.25999']
['speaker4_F', '152.25999', '156.28000']


Plutôt qu'un simple tableau, on voudrait obtenir un graphique de ce type:
![graphique](plot_1.png)

In [7]:
def preprocess_raw_speads_output(infile, outfile):
    with open(outfile, 'w') as dest:
        dest.write("ID\tGender\tStart\tEnd\n")
        with open(infile, 'r') as source:
            reader = csv.reader(source, delimiter='\t')
            for row in reader:
                dest.write('{}\t{}\t{}\t{}\n'.format(row[0][:-2], row[0][-1:], row[1], row[2]))

In [8]:
preprocess_raw_speads_output(infile='conv1_speads.tsv', outfile='out/conv1_processed.tsv')